<a href="https://colab.research.google.com/github/wheemin-2/25-1-ESAA/blob/main/0314_HW_Ensemble%2BRandom_Forest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **부스팅**

- 약한 학습기를 여러개 연결하여 강한 학습기를 만드는 앙상블 방법
- 아이디어 : 앞의 모델을 보완해가면서 예측기 학습
- AdaBoost, Gradient Boosting

## **에이다부스트 (AdaBoost)**

아이디어
* 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이면서 이전 예측기를 보완한 새로운 예측기를 생성
> 이로써 새로운 예측기는 학습하기 어려운 샘플에 점점 맞춰지게 됨

![AdaBoost](https://velog.velcdn.com/images/kyungmin1029/post/aa6e33a1-b27f-471b-bd63-260a0b082211/image.png)

* 첫 번째 분류기 학습 > 예측 > 잘못 분류된 훈련 샘플의 가중치를 높임
* 두 번째 분류기 학습 (업데이트된 가중치 사용) > 예측 > 가중치 업데이트

*경사하강법이랑 무엇이 다른가?*
 * 경사하강법은 비용함수를 최소화하는 목적으로 **하나의 예측기**에서 모델 파라미터를 조정하고, 에이다부스트는 앙상블에 **예측기를 추가**하는 방식이다. (따라서 예측기마다 다른 가중치를 지닌다.)

**에이다부스트 알고리즘**

![Ada eq1](https://velog.velcdn.com/images/kyungmin1029/post/613c6948-831d-415d-b210-fc7e676fa7c8/image.png)

- 각 샘플 가중치 $w^{(i)}$ : 초기에 1/m (샘플 수) 로 초기화
- 첫 번째 예측기 학습 > 가중치 적용된 에러율 $r_1$이 훈련 세트에 대해 계산됨

![Ada eq2](https://velog.velcdn.com/images/kyungmin1029/post/babdbf24-27e4-46da-90fa-ddc206554dd0/image.png)

- eta : 학습률 (default=1)
- 예측기가 정확할수록 가중치 ↑
    * error인 $r_j$값이 낮아지기 때문
    * 무작위로 예측하는 예측기라면 0에 가까울 것
    * 무작위 추측보다 정확도가 낮다면 가중치는 음수

![Ada eq3](https://velog.velcdn.com/images/kyungmin1029/post/019282d7-f59d-48d9-82eb-f2ef09c6d08f/image.png)

- i번째 각 샘플의 가중치를 업데이트함
- 예측이 맞으면 그대로, 틀리면 예측기의 가중치를 더 주는 식으로 계산
- 후에 샘플의 가중치를 정규화 (모든 가중치의 합 $\sum_{i=1}^{m} w^{(i)}$
으로 나눈다.)


**에이다부스트 예측**

![Ada eq4](https://velog.velcdn.com/images/kyungmin1029/post/8e8bb570-0eae-4bba-af1a-c71b947ac2e3/image.png)

- 단순히 모든 예측기의 예측을 계산 > 예측기 가중치 $\alpha_j$를 더해 예측 결과를 생성
- 가중치 합이 가장 큰 클래스가 예측 결과!

In [1]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

#5장에서 소개한 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X,y = make_moons(n_samples=100, noise=0.15)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm='SAMME', learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

## **그래디언트 부스팅**

**공통점)**
에이다부스트와 마찬가지로 예측기를 순차적으로 추가하는 방식

**차이점)**
에이다부스트는 반복마다 샘플의 가중치를 수정하는 반면, 그래디언트 부스팅은 이전 예측기가 만든 잔여 오차를 학습시킨 새로운 예측기를 추가

In [6]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [7]:
# 첫 번째 예측기에서 생긴 잔여 오차에 두번째 결정트리 훈련
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [8]:
# 두 번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델 훈련
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

**새로운 샘플에 대한 예측 생성**
: 모든 트리의 예측을 더함


```
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))
```


![GBRT](https://velog.velcdn.com/images/kyungmin1029/post/9fb97193-d1c6-4e1f-be50-6f08f05a4008/image.png)

In [9]:
# 사이킷런의 GradientBoostingRegressor 이용
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

- 학습률 : 각 트리의 기여 정도를 조절하는 파라미터
- 학습률이 낮으면 많은 트리가 필요하지만 충족될 경우 성능 ↑
- 작은 학습률로 훈련시킨 두 개의 GBRT 앙상블 중 왼쪽은 훈련 세트를 학습하기에는 트리가 충분하지 않은 반면 오른쪽은 트리가 너무 많아 훈련 세트에 과대적합

 ![learning_rate](https://velog.velcdn.com/images/kyungmin1029/post/fd6fa163-76cd-41fa-8476-0ab7a7c9d496/image.png)

**최적의 트리 수 찾기**

- 조기 종료 기법 사용
- staged_predict() 메서드 사용

In [10]:
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=66)

![early_stopping](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FF2VC2%2FbtqF3FuFoJI%2FLFvyMlvck0zNysWvFC8bEK%2Fimg.jpg)

In [11]:
# 실제로 훈련을 중지하는 방법
# warm_start=True

gbrt = GradientBoostingRegressor(max_depth=2,warm_start = True)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1,120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train,y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val,y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:   # 다섯 번 반복 동안 검증 오차가 향상 X
            break  #조기 종료

🤔 **확률적 그레디언트 부스팅**

각 트리가 훈련할 때 훈련 샘플의 비율을 subsample 매개변수로 주는 방법

- subsample = 0.25 이면 각 트리는 무작위로 25% 를 훈련 샘플을 선택해 학습
- 편향이 높아지고 분산은 낮아진다.

🤔 **XGBoost**

최적화된 그레이디언트 부스팅 구현 라이브러리
- 빠른 속도, 확장성, 이식성이 강점

In [12]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [13]:
# 조기종료
xgb_reg = xgboost.XGBRegressor(early_stopping_rounds = 2)
xgb_reg.fit(X_train, y_train,
            eval_set = [(X_test, y_test)])
y_pred = xgb_reg.predict(X_test)

mean_squared_error(y_test, y_pred)

[0]	validation_0-rmse:0.38623
[1]	validation_0-rmse:0.30334
[2]	validation_0-rmse:0.23005
[3]	validation_0-rmse:0.18788
[4]	validation_0-rmse:0.16174
[5]	validation_0-rmse:0.14716
[6]	validation_0-rmse:0.13550
[7]	validation_0-rmse:0.12940
[8]	validation_0-rmse:0.12736
[9]	validation_0-rmse:0.12695
[10]	validation_0-rmse:0.12332
[11]	validation_0-rmse:0.12116
[12]	validation_0-rmse:0.12022
[13]	validation_0-rmse:0.11984
[14]	validation_0-rmse:0.12135


0.014361301437020302

# **스태킹**

기본 아이디어

* 앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수(e.g. 직접 투표)를 사용하는 대신, 취합하는 모델을 훈련시킬 수는 없을까?

![stacking1](https://velog.velcdn.com/images/kyungmin1029/post/3f0744bc-943e-44b7-a339-26f9b72a65f0/image.png)

세 예측기는 각각 예측을 수행하고, 마지막 예측기(블렌더, 메타학습기)가 최종 예측을 만드는 구조로 구성

![stacking2](https://velog.velcdn.com/images/kyungmin1029/post/a328519b-46fa-4147-a2fa-5113ceb6dd9b/image.png)

- 훈련 세트를 두 개로 쪼개고, 첫번째 반쪽은 3개의 예측기를 훈련시키는 데 사용

![stacking3](https://velog.velcdn.com/images/kyungmin1029/post/e563741a-c7b3-45ae-9119-5c876290a012/image.png)

- 나머지 반쪽으로는 입력으로 주고 예측을 만듦
- 해당 예측은 다시 입력으로 블렌더에 들어감
- 반복하여 여러 레이어를 만들 수도 있다 (마치 신경망 처럼)

![stacking4](https://velog.velcdn.com/images/kyungmin1029/post/d54f86dd-935d-43ae-a5ae-b1763a79170c/image.png)